In [1]:
import geopandas as gpd
import requests
import json
import fiona
from shapely.geometry import Point

In [2]:
fiona.supported_drivers['KML'] = 'rw'

In [3]:
restriction_areas = gpd.read_file('flight_restriction_areas.kml', driver='KML')
restriction_areas

,Name,Description,geometry
0,Genova/Sestri 10/28 - 1,0,"POLYGON Z ((8.91680 44.42656 0.00000, 8.76924 ..."
1,Genova/Sestri 10/28 - 2,25,"POLYGON Z ((8.97015 44.43401 0.00000, 8.72227 ..."
2,Genova/Sestri 10/28 - 3,45,"POLYGON Z ((9.03816 44.45579 0.00000, 8.79744 ..."
3,Parco naturale regionale di Portofino,0,"POLYGON Z ((9.21908 44.32563 0.00000, 9.14994 ..."
4,Genova CTR,60,"POLYGON Z ((9.23428 44.42709 0.00000, 8.79741 ..."


In [ ]:
# Generate random points within each polygon
points = restriction_areas.sample_points(10)
points.explore(m=restriction_areas.explore(), color='red')

In [ ]:
# Parse the points coordinates to make a string for the request to open elevation API
def parse_points(points):
    coords = []
    msg = []

    for p in points:
        coords.append(list(p.coords))

    for i in range(len(coords)):
        coords[i] = [(x[1], x[0]) for x in coords[i]]
    
    for c in coords:
        msg.append(str(c[0][0]) + ',' + str(c[0][1]))
    
    msg = '|'.join(msg)

    return msg

In [ ]:
payloads = []
for p in points:
    payloads.append(parse_points(list(p.geoms)))
payloads


In [ ]:
def get_elevation(coords):
    api_url = 'https://api.open-elevation.com/api/v1/lookup?locations='
    url = api_url + coords
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if 'results' in data and len(data['results']) > 0:
            # Get all elevations and return
            # elevations = []
            # for result in data['results']:
            #     elevations.append(result['elevation'])
            return data
        else:
            print("No results found for the provided coordinates.")
    else:
        print("Error occurred while fetching data:", response.status_code)

In [ ]:
data = {}

for i in range(len(payloads)):
    data[restriction_areas['Name'][i]] = get_elevation(payloads[i])['results']

with open("elevations.json", "w") as outfile:
    outfile.write(json.dumps(data, indent=4))


In [ ]:
get_elevation(payloads[0])['results']